## Missing Tabular Data Enrichment With YOU Research API 

In this tutorial, we will enrich the missing values in a tabular dataset primarily using `You.com` research API. 

### Install required packages

In [9]:
%%capture
! pip install pandas
! pip install openai
! pip install openai
! pip install python-dotenv
! pip install requests

### Loading API keys

Set the environment variables `YDC_API_KEY` and `OPENAI_API_KEY` in a .env file which we will load using `dotenv`.

In [8]:
import dotenv
dotenv.load_dotenv(".env", override=True)

True

### Creating a synthetic dataset for enrichment

We will create a dataframe with a few company domains and columns such as number of employees, NAICS code, headquarter and year founded with missing values. Our aim is to fill these missing values using `You.com` research API.  

In [9]:
import pandas as pd

data = {
    'company': ["Apple", 'Canadian Tire', 'Home Depot', 'LinkedIn Corporation', 'General Dynamics'],
    'number of employees': ['', '', '', '', ''],
    'NAICS code(s)': ['', '', '', '', ''],
    'headquarter location': ['', '', '', '', ''],
    'founded year': ['', '', '', '', ''],
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
df.head()

,company,number of employees,NAICS code(s),headquarter location,founded year
0,Apple,,,,
1,Canadian Tire,,,,
2,Home Depot,,,,
3,LinkedIn Corporation,,,,
4,General Dynamics,,,,


### Searching and getting results with `You.com` API

We will obtain the missing values with a simple call to `You.com` API

In [12]:
# TODO - Switch out to use langchain/dspy implementation instead of reuests once available

import requests
import os

# obtain the API key from the environment
headers = {'x-api-key': os.environ['YDC_API_KEY']}

def get_research_data(company_name, missing_cols, mode="research"):
    """
    Run the query with the given company domain and missing columns to obtain the missing data
    in the form of a JSON object.
    """
    endpoint = f"https://chat-api.you.com/{mode}"
    params = {"query": f"""
              I am trying to find some missing information corresponding to {company_name}.
              For these companies, get me more information on each of the following {missing_cols}.
              """
        }
    response = requests.get(endpoint, params=params, headers=headers)
    return response.json()["answer"]

In [13]:
# Let's look at the results returned by the API for one of the companies
you_response = get_research_data("General Dynamics",
                           ['number_of_employees', 'NAICS_code(s)', 'headquarter_location', 'founded_year'])
print(you_response)

# Research Report on General Dynamics

## Overview
General Dynamics is a prominent American aerospace and defense corporation. This report provides detailed information on the number of employees, NAICS code(s), headquarter location, and the year the company was founded.

## Number of Employees
As of December 31, 2023, General Dynamics had 111,600 employees [[1]](https://stockanalysis.com/stocks/gd/employees/#:~:text=General%20Dynamics%20had%20111%2C600%20employees%20on%20December%2031%2C%202023)[[2]](https://www.forbes.com/companies/general-dynamics/#:~:text=Employees%20111%2C600). This represents an increase of 5,100 employees or 4.79% compared to the previous year [[3]](https://stockanalysis.com/stocks/gd/employees/#:~:text=The%20number%20of%20employees%20increased%20by%205%2C100%20or%204.79%25%20compared%20to%20the%20previous%20year). The company employs over 100,000 people across 43 countries [[4]](https://leadiq.com/c/general-dynamics/5a1d95922300005a008441e4/employee-directory#:

### Value extraction with GPT function calling

The response from You.com API is in long text format along with the relevant links. As we want to fill our table with the exact information, we will be using chatGPT funtion calling to extract the information for each of the missing fields. 

In [14]:
from openai import OpenAI
import json

client = OpenAI()

In [16]:
# Let's create a function to extract the missing data from the You.com response using chatGPT function calling
def get_missing_data(you_com_response):
    """
    Given the response from the You.com API, extract the missing data fields using function calling with chatGPT
    and return the extracted data as a json object. 
    """
    prompt = f"Given the {you_com_response}, extract number of employees, NAICS code, headquarter location, and founded year."

    # function description to specify the format of the extracted data
    function = [
        {
            "name": "get_company_data",
            "description": "Extract the relevant data corresponding to each field",
            "parameters": {
                "type": "object",
                "properties": {
                    "number of employees": {
                        "type": "string",
                        "description": "The number of employees working in the company",
                    },
                    "NAICS code(s)": {
                        "type": "string",
                        "description": "The NAICS code of the company",
                    },
                    "headquarter location": {
                        "type": "string",
                        "description": "The headquarter location of the company",
                    },
                    "founded year": {
                        "type": "string",
                        "description": "The year in which the company was founded",
                    },
                },
                "required": ["number of employees", "NAICS code", "headquarter location", "founded year"],
            },
        }
    ]

    # call the chatGPT model with the function calling
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ],
        functions=function,
        function_call="auto"
    )

    # get the message from the completion which contains the extracted data
    output = completion.choices[0].message
    # parse the output to extract the json object
    json_output = json.loads(output.function_call.arguments)
    
    return json_output

In [18]:
# Let's look at the extracted missing data for the company from earlier
get_missing_data(you_response)

{'number of employees': '111600',
 'NAICS code(s)': '336414, 336411',
 'headquarter location': 'Reston, Virginia',
 'founded year': '1952'}

### Fill the missing data

Finally, let's run our functions to first search and get the infromation using `You.com` research API then extract the values of the missing columns in `json` format with GPT function calling.

In [224]:
# call the API to get the missing data and update the DataFrame
for index, row in df.iterrows():
    missing_cols = [col for col in df.columns if row[col] == '']

    if missing_cols:
        # get the missing data from the You.com API
        you_response = get_research_data(row['company'], missing_cols)

        # extract the missing data using chatGPT function calling
        json_response = get_missing_data(you_response)

        # fill the data into original DataFrame
        for col in missing_cols:
            df.at[index, col] = json_response.get(col, '')

In [225]:
# Finally, let's display the updated DataFrame with the missing data filled in
df.head()

,company,number of employees,NAICS code(s),headquarter location,founded year
0,Apple,"161,000","511210, 334111","Cupertino, California, 95014, United States","April 1, 1976"
1,Canadian Tire,"68,000","441310, 441320, 441110, 441120, 441210, 441222...","Toronto, Ontario",1922
2,Home Depot,"463,100","444130, 444110, 23","2455 Paces Ferry Road, Atlanta, Georgia 30339,...",1978
3,LinkedIn Corporation,"19,400",541511,"Sunnyvale, California",2003
4,General Dynamics,"111,600",336414,"Reston, Virginia, USA",1952
